Me trying to learn deep learning with tensorflow

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [6]:
#Dataset to be used
#The dataset is basically a bunch of handwritten numbers
mnist = tf.keras.datasets.mnist
#x-train and x-test are appart of the training set and y-test and y-train are appart of the test set
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [9]:
#One epoch is the time step that is incremented every time it has went through all the samples in the training set.
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 9s 157us/sample - loss: 0.2961 - accuracy: 0.9140
Epoch 2/5
60000/60000 [==============================] - 8s 139us/sample - loss: 0.1419 - accuracy: 0.9577
Epoch 3/5
60000/60000 [==============================] - 8s 140us/sample - loss: 0.1093 - accuracy: 0.9658
Epoch 4/5
60000/60000 [==============================] - 8s 137us/sample - loss: 0.0884 - accuracy: 0.9734
Epoch 5/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.0754 - accuracy: 0.9765
10000/1 - 1s - loss: 0.0407 - accuracy: 0.9749


[0.08049258911972866, 0.9749]

In [10]:
#Computer's prediction
print(y_test)

[7 2 1 ... 4 5 6]


This is me trying to figure out how predictive models work

In [61]:
import pandas as pd
#use one-word names for columns or it will give you an error
CSV_COLUMN_NAMES = ['lengthinfeet', 'weightintons', 'speedmph', 'biteforcenewtons', 'Species']
SPECIES = ['T-rex', 'Megalodon', 'Giganotosaurus']
#Make sure the 
train = pd.read_csv('Dino_train.csv', names=CSV_COLUMN_NAMES, skipinitialspace=True, skiprows=1, engine="python")
test = pd.read_csv('Dino_test.csv', names=CSV_COLUMN_NAMES, skipinitialspace=True, skiprows=1, engine="python")
train.head()

,lengthinfeet,weightintons,speedmph,biteforcenewtons,Species
0,40.0,7.00,18.0,57000,0
1,21.0,60.00,20.0,182000,1
2,43.0,8.60,31.0,3340,2
3,42.6,9.75,17.0,56900,0
4,44.0,50.00,19.0,156001,1


In [62]:
train_y = train.pop('Species')
test_y = test.pop('Species')

def input_evaluation_set():
    features = {'lengthinfeet': np.array([40.0, 21.0]),
                'weightintons':  np.array([7.00, 60.00]),
                'speedmph': np.array([18.0, 20.0]),
                'biteforcenewtons':  np.array([57000, 182000])}
    labels = np.array([2, 1])
    return features, labels
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\WALLAC~1\\AppData\\Local\\Temp\\tmpla3umzp6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000017F48513908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [63]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

eval_result = classifier.evaluate(
input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\WALLAC~1\AppData\Local\Temp\tmpla3umzp6\model.ckpt.
INFO:tensorflow:loss = 2334.2407, step = 0
INFO:tensorflow:global_step/sec: 266.421
INFO:tensorflow:loss = 119.55055, step = 100 (0.375 sec)
INFO:tensorflow:global_

In [64]:
# Generate predictions from the model
expected = ['T-rex', 'Megalodon', 'Giganotosaurus']
predict_x = {
    'lengthinfeet': [41.5, 23, 44],
    'weightintons': [7.1, 59.3, 8.4],
    'speedmph': [18.6, 20.6, 30],
    'biteforcenewtons': [57023, 182023, 3342],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\WALLAC~1\AppData\Local\Temp\tmpla3umzp6\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "T-rex" (95.9%), expected "T-rex"
Prediction is "Megalodon" (99.3%), expected "Megalodon"
Prediction is "Giganotosaurus" (99.9%), expected "Giganotosaurus"
